## Combine autorigging outputs for evaluation
### Load & define functions

In [ ]:
%load_ext autoreload
%autoreload 2
key=0
import sys
sys.path.append('..')

import os
from os import listdir
from os.path import join
import torch
import numpy as np

import open3d as o3d

from utils.obj_utils import ObjLoader, face2edge
from utils.bvh_utils import bvh_to_JMBM
from utils.skin_util import vertex_transform

In [ ]:
from utils.joint_util import joint_names, maketree

tree = maketree(22)
hier_str_list = []
for node_idx, child_list in enumerate(tree):
    for child_idx in child_list:
        hier_str = 'hier %s %s'%(joint_names[node_idx], joint_names[child_idx])
        hier_str_list.append(hier_str)

        
def combine_and_save_log(log_name, joint_result, skin_result, threshold=0.2):
    
    ## Filter skin weight
    def filter_weights(vertex_weights:list, threshold=0.2):
        weight_sum = 0
        for idx in range(len(vertex_weights)):
            if vertex_weights[idx] < threshold:
                vertex_weights[idx] = 0
            else:
                weight_sum += vertex_weights[idx]
        for idx in range(len(vertex_weights)):
            vertex_weights[idx] /= weight_sum

    joint_str_list = []
    for joint_index, joint_pos in enumerate(joint_result):
        joint_str = 'joints %s %f %f %f'%(joint_names[joint_index], joint_pos[0], joint_pos[1], joint_pos[2])
        joint_str_list.append(joint_str)

    vertex_str_list = []
    for vertex_index, vertex_weights in enumerate(skin_result):
        filter_weights(vertex_weights, 0.2)
        v_weights = np.array(vertex_weights)
        indices = v_weights.argsort()[-3:][::-1]
        indices = [idx for idx in indices if v_weights[idx] > 0]
        vertex_str = 'skin %d '%(vertex_index) + ' '.join(["%s %f"%(joint_names[idx], v_weights[idx]) for idx in indices])
        vertex_str_list.append(vertex_str)


    str_list = joint_str_list + ['root Hips'] + vertex_str_list + hier_str_list
    with open(log_name, 'w') as f:
        for str_line in str_list:
            f.write("%s\n" % str_line)

### Path for Dataset and Logs

In [ ]:
data_dir = "../data/mixamo"

joint_pos_result_dir = '../logs/vox/vox_ori_all_s4.5_HG_mean_stack2_down2_lr3e-4_b4_ce/test'
joint_rot_result_dir = '../logs/bvh/bvh_all_lr1e-3_zeroroot_bn/test_709'
skin_result_dir = '../logs/skin/b4_tpl_and_euc0.12_lr1e-4_bn_epoch30/test'

assert(os.path.exists(joint_pos_result_dir))
assert(os.path.exists(joint_rot_result_dir))
assert(os.path.exists(skin_result_dir))

### Combine Network Outputs from Logs

In [ ]:
characters_list = sorted(listdir(joint_pos_result_dir))

joint_results = listdir(join(joint_pos_result_dir, characters_list[0]))
motions_list = sorted([result.split('_joint.npy')[0] for result in joint_results if result.endswith('_joint.npy')])
print(len(characters_list), len(motions_list))

for character in characters_list:
    
    # Load ground truth skin
    skin_gt = np.genfromtxt(join(data_dir, 'weights', '%s.csv'%(character)), delimiter=',')
    print('\nReading Character: ', character)

    for motion in motions_list:
        
        ### Load Result for each character & motion
        print('Reading Motion: ', motion, end='\r')
        
        ## Load Joint Position Result
        joint_pos_file = join(joint_pos_result_dir, '%s/%s_joint.npy'%(character, motion))
        joint_pos = np.load(joint_pos_file, allow_pickle=True).item()['pred_heatmap_joint_pos_mask']
        
        ## Load Joint Roatation Result
        joint_rot_file = join(joint_rot_result_dir, '%s/%s_pred_rot.npy'%(character, motion))
        joint_rot = np.load(joint_rot_file)
                
        ## Load Skin Result 
        skin_file = os.path.join(skin_result_dir, '%s/%s_skin.csv'%(character, motion))
        skin = np.loadtxt(skin_file, delimiter=',') # (V, 22)
        
    
        ### Convert to T-pose
        ## Load original mesh
        obj = ObjLoader(join(data_dir, 'objs_fixed', '%s/%s.obj'%(character, motion)),skip_face=False)
        vertex = np.array(obj.vertices)

        mesh = o3d.io.read_triangle_mesh(join(data_dir, 'objs_fixed', '%s/%s.obj'%(character, motion)))
        vertex = np.asarray(mesh.vertices)
        o3d.io.write_triangle_mesh('tmp.obj', mesh)

        ## Compute JM, BM
        JM4x4_glob_pred, BM4x4_pred = bvh_to_JMBM(global_joint_pos= joint_pos, global_rotation_matrices= joint_rot,
                                          bindpose_root_offset=[0,0,0], JM_type='glob')
        ## Convert vertices to T-pose
        vertex_pred = vertex_transform(torch.from_numpy(vertex).float(), torch.from_numpy(JM4x4_glob_pred).float(),
                                              torch.from_numpy(BM4x4_pred).float(), torch.from_numpy(skin).float(), JM_type='glob')[0]
        vertex_gtskin = vertex_transform(torch.from_numpy(vertex).float(), torch.from_numpy(JM4x4_glob_pred).float(),
                                              torch.from_numpy(BM4x4_pred).float(), torch.from_numpy(skin_gt).float(), JM_type='glob')[0]
        

        #### SAVE #### 
        os.makedirs('../result', exist_ok=True)
        
        ## Save T-pose vertex

        np.save('../result/%s_%s_vertex_pred.txt'%(character, motion), vertex_pred)
        np.save('../result/%s_%s_vertex_gtskin.txt'%(character, motion), vertex_gtskin)
        
        ## Save T-pose converted obj
        mesh_pred = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(vertex_pred), mesh.triangles)
        mesh_gtskin = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(vertex_gtskin), mesh.triangles)
        o3d.io.write_triangle_mesh('../result/%s_%s_vertex_pred.obj'%(character, motion), mesh_pred)
        o3d.io.write_triangle_mesh('../result/%s_%s_vertex_gtskin.obj'%(character, motion), mesh_gtskin)

        ## Save "mesh + joint_pos + skin" for evaluation format (NOT in T-pose)
        combine_and_save_log('../result/%s_%s_rig.txt'%(character, motion),
                             joint_pos, skin, threshold=0.2)
        
        ## Save "mesh + joint_pos + skin" for evaluation format (in T-pose)
        joint_pos_T = BM4x4_pred[:,:3,3]
        combine_and_save_log('../result/%s_%s_T-rig.txt'%(character, motion),
                             joint_pos_T, skin, threshold=0.2)
 
        ### SAVE ALL if you need it ###
        if True:
            np.save('../result/%s_%s_combined'%(character, motion),{
                     'joint_pos': joint_pos,
                     'joint_rot': joint_rot,
                     'skin': skin,
                     'BM4x4_pred': BM4x4_pred,
                     'vertex_pred': vertex_pred, # vertex transformed with pred skin
                     'vertex_gtskin': vertex_gtskin, # vertex transformed with gt skin
                     'origin': (joint_pos_result_dir, joint_rot_result_dir, skin_result_dir)
                    })
print("\nFinished")